In [7]:
import sqlite3
connection = sqlite3.connect("Food_Data.db")
cursor = connection.cursor()   

In [9]:
#  Creating the provider table if it doesn't exist
cursor.execute("""
    CREATE TABLE IF NOT EXISTS providers (
        Provider_ID INT,
        Name VARCHAR(255),
        Type VARCHAR(100),
        Address TEXT,
        City VARCHAR(100),
        Contact VARCHAR(50)
    )
""")
connection.commit()


In [31]:
import pandas as pd

# Load the data into a pandas DataFrame
providers = pd.read_csv("D:/Data Science/project/Food waste management system/providers_data.csv")
receivers = pd.read_csv("D:/Data Science/project/Food waste management system/receivers_data.csv")
claims_data = pd.read_csv("D:/Data Science/project/Food waste management system/claims_data.csv")
food_listings_data = pd.read_csv("D:/Data Science/project/Food waste management system/food_listings_data.csv")


In [21]:
# Insert data row by row
for index, row in providers.iterrows():
    cursor.execute(
        "INSERT INTO providers (Provider_ID, Name, Type, Address, City, Contact) VALUES (?, ?, ?, ?, ?, ?)",
        (
            row['Provider_ID'],
            row['Name'],
            row['Type'],
            row['Address'],
            row['City'],
            row['Contact']
        )
    )

connection.commit()

In [19]:
#How many food providers are there in each city?

query = """
    SELECT City, COUNT(*) AS Provider_Count
    FROM providers
    GROUP BY City
    ORDER BY Provider_Count DESC;
"""

cursor.execute(query)
result = cursor.fetchall()

# Convert result into a DataFrame for better readability
df = pd.DataFrame(result, columns=["City", "Provider_Count"])
df


,City,Provider_Count
0,South Christopherborough,3
1,New Carol,3
2,Williamview,2
3,West Lauraborough,2
4,West Christopher,2
...,...,...
958,Alexanderchester,1
959,Aguirreville,1
960,Adamsville,1
961,Adamsview,1


In [21]:
#  Create the receivers table if it doesn't exist
cursor.execute("""
    CREATE TABLE IF NOT EXISTS receivers (
        Receiver_ID INT,
        Name VARCHAR(255),
        Type VARCHAR(100),
        City VARCHAR(100),
        Contact VARCHAR(50)
    )
""")
connection.commit()

In [23]:
#  Insert data row by row
for index, row in receivers.iterrows():
    cursor.execute(
        "INSERT INTO receivers (Receiver_ID, Name, Type, City, Contact) VALUES (?, ?, ?, ?, ?)",
        (
            row['Receiver_ID'],
            row['Name'],
            row['Type'],
            row['City'],
            row['Contact']
        )
    )

connection.commit()

In [24]:
#1. How many receivers are there in each city?
query = """
    SELECT City, COUNT(*) AS Receiver_Count
    FROM receivers
    GROUP BY City
    ORDER BY Receiver_Count DESC;
"""

cursor.execute(query)
result = cursor.fetchall()

# Convert result into a DataFrame for better readability
df = pd.DataFrame(result, columns=["City", "Receiver_Count"])
df

,City,Receiver_Count
0,New Christopher,3
1,Williamsfort,2
2,West Richard,2
3,West James,2
4,Stevenmouth,2
...,...,...
961,Alexanderbury,1
962,Aguilarstad,1
963,Aguilarbury,1
964,Adamland,1


In [18]:
#Which type of food provider (restaurant, grocery store, etc.) contributes the most food?
query1 = '''
SELECT Type, COUNT(Type) AS Most_Food_Provider
FROM providers
GROUP BY Type
ORDER BY Most_Food_Provider DESC;
'''
cursor.execute(query1)
result = cursor.fetchall()

# Convert result into DataFrame 
df = pd.DataFrame(result, columns=['Type', 'Most_Food_Provider'])
df

,Type,Most_Food_Provider
0,Supermarket,262
1,Grocery Store,256
2,Restaurant,246
3,Catering Service,236


In [30]:
# What is the contact information of food providers in a specific city?
query = '''
SELECT City, Contact 
FROM providers
where City = 'South Jeffrey';
'''
cursor.execute(query)
result = cursor.fetchall()

#converting result into a dataframe
df = pd.DataFrame(result, columns= ['City','Contact'])
df

,City,Contact
0,South Jeffrey,+1-342-496-7540


In [39]:
#  Create the claims table if it doesn't exist
cursor.execute("""
    CREATE TABLE IF NOT EXISTS claims_data (
        Claim_ID INT PRIMARY KEY,
        Food_ID INT,
        Receiver_ID INT,
        Status VARCHAR(50),
       Timestamp DATETIME,
       FOREIGN KEY (Food_ID) REFERENCES food_listing (Food_ID)
       FOREIGN KEY (Receiver_ID) REFERENCES receivers (Receiver_ID)
    )
""")
connection.commit()

In [40]:
# Insert data row by row
for index, row in claims_data.iterrows():
    cursor.execute(
        "INSERT INTO claims_data (Claim_ID, Food_ID, Receiver_ID, Status, Timestamp) VALUES (?, ?, ?, ?, ?)",
        (
            row['Claim_ID'],
            row['Food_ID'],
            row['Receiver_ID'],
            row['Status'],
            row['Timestamp']
        )
    )

connection.commit()

In [52]:
#create food listing table if not exist
cursor.execute("""
    CREATE TABLE IF NOT EXISTS food_listings_data (
       Food_ID INT PRIMARY KEY,
       Food_Name VARCHAR(100),
       Quantity INT,
       Expiry_Date DATETIME,
       Provider_ID INT,
       Provider_Type VARCHAR(100),
       Location VARCHAR(100),
       Food_Type VARCHAR(100),
       Meal_Type VARCHAR(100),
       FOREIGN KEY (Provider_ID) REFERENCES prividers(Provider_ID)
    )
""")
connection.commit()

In [54]:
# Insert data row by row
for index, row in food_listings_data.iterrows():
    cursor.execute(
        "INSERT INTO food_listings_data (Food_ID, Food_Name, Quantity, Expiry_Date, Provider_ID, Provider_Type, Location, Food_Type, Meal_Type) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)",
        (
            row['Food_ID'],
            row['Food_Name'],
            row['Quantity'],
            row['Expiry_Date'],
            row['Provider_ID'],
            row['Provider_Type'],
            row['Location'],
            row['Food_Type'],
            row['Meal_Type']
         
        )
    )

connection.commit()

In [63]:
#Which receivers have claimed the most food?
query = '''
SELECT Receiver_ID, COUNT(*) AS Total_claims
FROM claims_data
WHERE status = 'Completed'
GROUP BY Receiver_ID
ORDER BY Total_claims DESC
LIMIT 1;
'''
cursor.execute(query)
result = cursor.fetchall()

#converting result into a dataframe
df = pd.DataFrame(result, columns= ['Receiver_ID','Total_claims'])
df

,Receiver_ID,Total_claims
0,505,3


In [64]:
# What is the total quantity of food available from all providers?
query = '''
SELECT  SUM(Quantity) AS Total_Qnatity_Of_Food
FROM food_listings_data;
'''
cursor.execute(query)
result = cursor.fetchall()

#converting result into a dataframe
df = pd.DataFrame(result, columns= ['Total_Qnatity_Of_Food'])
df

,Total_Qnatity_Of_Food
0,25794


In [70]:
# Which city has the highest number of food listings?
query = '''
SELECT Location, Count(*)  as Top_food_listing_city
FROM food_listings_data
GROUP BY Location
ORDER BY Top_food_listing_city DESC
LIMIT 1;
'''
cursor.execute(query)
result= cursor.fetchall()

#converting result into dataframe
df= pd.DataFrame(result, columns = ['City','Top_food_listing_city'])
df


,City,Top_food_listing_city
0,South Kathryn,6


In [71]:
#What are the most commonly available food types?

query = '''
SELECT Food_Name, Count(*)  as Common_Foods
FROM food_listings_data
GROUP BY Quantity
ORDER BY Common_foods DESC
LIMIT 5;
'''
cursor.execute(query)
result= cursor.fetchall()

#converting result into dataframe
df= pd.DataFrame(result, columns = ['Food_Name','Common_Foods'])
df

,Food_Name,Common_Foods
0,Salad,28
1,Pasta,27
2,Rice,27
3,Pasta,26
4,Rice,25


In [76]:
#How many food claims have been made for each food item?
query = '''
SELECT COUNT(*) AS Food_Claims, food_listings_data.Food_Name
FROM claims_data
LEFT JOIN food_listings_data
  ON claims_data.Food_ID = food_listings_data.Food_ID
GROUP BY food_listings_data.Food_Name
ORDER BY Food_Claims DESC;
'''
cursor.execute(query)
result = cursor.fetchall()

# Corrected column name (extra space removed)
df = pd.DataFrame(result, columns=['Food_Claims', 'Food_Name'])
df

,Food_Claims,Food_Name
0,122,Rice
1,114,Soup
2,110,Dairy
3,108,Fish
4,106,Salad
5,102,Chicken
6,94,Bread
7,87,Pasta
8,86,Vegetables
9,71,Fruits


In [79]:
#Which provider has had the highest number of successful food claims?
query = '''
SELECT food_listings_data.Provider_ID, COUNT(*) AS Successful_food_claims
FROM claims_data
LEFT JOIN food_listings_data
  ON claims_data.Food_ID = food_listings_data.Food_ID
WHERE claims_data.Status = 'Completed'
GROUP BY food_listings_data.Provider_ID
ORDER BY Successful_food_claims DESC
LIMIT 1;
'''
cursor.execute(query)
result = cursor.fetchall()

df = pd.DataFrame(result, columns=['Provider_ID', 'Successful_food_claims'])
df

,Provider_ID,Successful_food_claims
0,709,5


In [80]:
#What percentage of food claims are completed vs. pending vs. canceled?
query = '''
SELECT Status, 
       COUNT(*) * 100.0 / (SELECT COUNT(*) FROM claims_data) AS Percentage
FROM claims_data
GROUP BY Status;
'''
cursor.execute(query)
result = cursor.fetchall()

# Convert to DataFrame with proper columns
df = pd.DataFrame(result, columns=['Status', 'Percentage'])
df

,Status,Percentage
0,Cancelled,33.6
1,Completed,33.9
2,Pending,32.5


In [82]:
# What is the average quantity of food claimed per receiver?
query='''
SELECT claims_data.Receiver_ID, 
       AVG(food_listings_data.Quantity) AS Average_quantity_of_Food_Claimed
FROM claims_data
LEFT JOIN food_listings_data
  ON claims_data.Food_ID = food_listings_data.Food_ID
GROUP BY claims_data.Receiver_ID;
'''
cursor.execute(query)
result = cursor.fetchall()

# Convert to DataFrame
df = pd.DataFrame(result, columns=['Receiver_ID', 'Average_quantity_of_Food_Claimed'])
df

,Receiver_ID,Average_quantity_of_Food_Claimed
0,1,35.5
1,2,47.0
2,3,26.0
3,4,30.5
4,6,15.0
...,...,...
619,995,35.0
620,996,24.5
621,997,25.5
622,999,42.0


In [85]:
#Which meal type (breakfast, lunch, dinner, snacks) is claimed the most?
query = '''
SELECT food_listings_data.Meal_Type, 
       COUNT(*) AS Most_Claimed_Meal
FROM claims_data
INNER JOIN food_listings_data
  ON claims_data.Food_ID = food_listings_data.Food_ID
GROUP BY food_listings_data.Meal_Type
ORDER BY Most_Claimed_Meal DESC
LIMIT 1;
'''
cursor.execute(query)
result = cursor.fetchall()

# Convert to DataFrame
df = pd.DataFrame(result, columns=['Meal_Type', 'Most_Claimed_Meal'])
df

,Meal_Type,Most_Claimed_Meal
0,Breakfast,278


In [88]:
#What is the total quantity of food donated by each provider?
query= '''
SELECT providers.Name, providers.Provider_ID, SUM(food_listings_data.Quantity) As Total_Quantity
FROM providers
LEFT JOIN food_listings_data
ON providers.Provider_ID = food_listings_data.Provider_ID
GROUP BY providers.Provider_ID
ORDER BY Total_Quantity DESC;
'''
cursor.execute(query)
result= cursor.fetchall()

# convert it to a DataFrame
df = pd.DataFrame(result, columns= ['Name','Provider_ID','Total_Quantity'])
df

,Name,Provider_ID,Total_Quantity
0,Barry Group,709,179.0
1,"Evans, Wright and Mitchell",306,158.0
2,Smith Group,655,150.0
3,Nelson LLC,315,142.0
4,Ruiz-Oneal,678,140.0
...,...,...,...
995,"Parker, Flores and Jennings",13,NaN
996,Miller Ltd,12,NaN
997,Curtis-Lewis,8,NaN
998,Lawson-Walters,6,NaN
